In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

In [6]:
# ---------------------- 1. Veri Seti Yükleme ve Sütun Adları ----------------------
columns = [
    "id", "diagnosis",
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave_points_mean", "symmetry_mean", "fractal_dimension_mean",
    "radius_se", "texture_se", "perimeter_se", "area_se", "smoothness_se", "compactness_se",
    "concavity_se", "concave_points_se", "symmetry_se", "fractal_dimension_se",
    "radius_worst", "texture_worst", "perimeter_worst", "area_worst", "smoothness_worst",
    "compactness_worst", "concavity_worst", "concave_points_worst", "symmetry_worst", "fractal_dimension_worst"
]

df = pd.read_csv("wdbc.csv", names=columns, header=None)

# ---------------------- 2. Etiket Kodlama ----------------------
df["diagnosis"] = df["diagnosis"].map({'B': 0, 'M': 1})


In [7]:
# ---------------------- 3. Korelasyonu Yüksek Özellikleri Bulma ----------------------
# Korelasyon matrisini hesaplayalım
korelasyon_matrix = df.iloc[:, 2:].corr().abs()  # Hedef ve ID hariç sayısal sütunlar

# Üst üçgeni almak için
upper = korelasyon_matrix.where(np.triu(np.ones(korelasyon_matrix.shape), k=1).astype(bool))

# Korelasyonu yüksek olan özellikleri çıkarma
yuksek_korelasyon_ozellikler = [column for column in upper.columns if any(upper[column] > 0.9)]

# Yüksek korelasyonlu özellikleri yazdırma
print("Yüksek Korelasyonlu Özellikler:", yuksek_korelasyon_ozellikler)

Yüksek Korelasyonlu Özellikler: ['perimeter_mean', 'area_mean', 'concave_points_mean', 'perimeter_se', 'area_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'concave_points_worst']


In [10]:
# ---------------------- 4. Giriş/Çıkış Ayırma ----------------------
X = df.drop(["id", "diagnosis"], axis=1)
y = df["diagnosis"]

# ---------------------- 5. Eğitim/Test Bölme ve Ölçekleme ----------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ---------------------- 6. Yapay Sinir Ağı Modeli Oluşturma ----------------------
model = Sequential()
model.add(Input(shape=(X_train_scaled.shape[1],)))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(
    monitor='val_loss',       # val_loss değişimini izle
    patience=10,              # 5 epoch boyunca gelişme olmazsa durdur
    restore_best_weights=True # En iyi ağırlıklara geri dön
)
# ---------------------- 7. Modeli Eğit ----------------------
model.fit(X_train_scaled, y_train, epochs=50, validation_split=0.1, callbacks=[early_stop])

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.3675 - loss: 0.7665 - val_accuracy: 0.2000 - val_loss: 0.7937
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3723 - loss: 0.7221 - val_accuracy: 0.3000 - val_loss: 0.7414
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4125 - loss: 0.6846 - val_accuracy: 0.4500 - val_loss: 0.6932
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6311 - loss: 0.6362 - val_accuracy: 0.5750 - val_loss: 0.6465
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7616 - loss: 0.6050 - val_accuracy: 0.8000 - val_loss: 0.6049
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8460 - loss: 0.5704 - val_accuracy: 0.8250 - val_loss: 0.5665
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8843 - loss: 0.5431 - val_accuracy: 0.8750 - val_loss: 0.5315
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8942 - loss: 0.5196 - val_accuracy: 0.9000 - val_loss

In [11]:

# ---------------------- 8. Test ve Değerlendirme ----------------------
y_pred_prob = model.predict(X_test_scaled).flatten()
y_pred_class = (y_pred_prob > 0.45).astype(int)

acc = accuracy_score(y_test, y_pred_class)
f1 = f1_score(y_test, y_pred_class)
auc = roc_auc_score(y_test, y_pred_prob)

print("\n🧠 Yapay Sinir Ağı Performansı:")
print(f"✅ Doğruluk (Accuracy): {acc:.4f}")
print(f"🎯 F1 Skoru: {f1:.4f}")
print(f"📈 ROC AUC: {auc:.4f}")

print("\n📋 Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred_class, target_names=["Benign", "Malignant"]))


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

🧠 Yapay Sinir Ağı Performansı:
✅ Doğruluk (Accuracy): 0.9942
🎯 F1 Skoru: 0.9921
📈 ROC AUC: 0.9987

📋 Sınıflandırma Raporu:
              precision    recall  f1-score   support

      Benign       0.99      1.00      1.00       107
   Malignant       1.00      0.98      0.99        64

    accuracy                           0.99       171
   macro avg       1.00      0.99      0.99       171
weighted avg       0.99      0.99      0.99       171



In [34]:

# burada modelin testi test verileriyle yapılmıştır

# Tahmin olasılıkları (0-1 arasında)
y_pred_prob = model.predict(X_test_scaled)

# Sınıf tahminleri (0: benign, 1: malignant)
y_pred_class = (y_pred_prob > 0.45).astype(int)
import pandas as pd

# Gerçek etiketlerle birlikte DataFrame'e dönüştürelim
results = pd.DataFrame({
    'Gerçek': y_test.values,
    'Tahmin': y_pred_class.flatten(),
    'Olasılık(%)': y_pred_prob.flatten()*100 
})

# İlk 10 tahmini gösterelim
print(results.head(40))


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
    Gerçek  Tahmin  Olasılık(%)
0        0       0     9.556339
1        0       0     4.186938
2        0       0     4.134171
3        0       0     4.180023
4        0       0     4.721162
5        0       0     4.161822
6        1       1    96.941360
7        0       0     4.362566
8        1       1    96.315727
9        0       0     4.180187
10       0       0     4.434204
11       1       1    96.998032
12       0       0     4.148760
13       0       0     4.144710
14       0       0     4.135836
15       0       0     5.132816
16       0       0     4.169230
17       1       1    96.981743
18       1       1    96.998360
19       1       1    96.967720
20       1       1    96.996727
21       1       1    57.481503
22       1       1    96.995651
23       0       0     4.138478
24       0       0     4.305354
25       1       1    96.421486
26       1       1    96.975128
27       1       1    96.988335
28       0       0     4.162274
29